In [8]:
! jupyter nbextensions_configurator enable --user






Enabling: jupyter_nbextensions_configurator
- Writing config: /home/arnaud/.jupyter
    - Validating...
      jupyter_nbextensions_configurator 0.4.1 OK
Enabling notebook nbextension nbextensions_configurator/config_menu/main...
Enabling tree nbextension nbextensions_configurator/tree_tab/main...


In [5]:
import pandas as pd

In [8]:
d={'Name': ['Class Balanced threshold', 'Oversampling threshold','Undersampling threshold','Class Balanced','Oversampling','Undersampling']
    , 'score': [0.015534301867066854, 0.013039691792487273,0.0, -0.0020701433944061067, -0.00957380312449455,-0.05678670011795548]}

In [10]:
df = pd.DataFrame(data=d)
df.sort_values(by:='score',ascending=False)

,Name,score
0,Class Balanced threshold,0.015534
1,Oversampling threshold,0.013040
2,Undersampling threshold,0.000000
3,Class Balanced,-0.002070
4,Oversampling,-0.009574
5,Undersampling,-0.056787


In [8]:
d = {'AMT_CREDIT': [1000, 2000,3000,4000,6000], 'y_actual': [0, 1, 0, 1,0]}
df = pd.DataFrame(data=d)
df['max_gain']=0
df.loc[df['y_actual']==0,'max_gain']=df.loc[df['y_actual']==0,'AMT_CREDIT']#*0.02
df

,AMT_CREDIT,y_actual,max_gain
0,1000,0,1000
1,2000,1,0
2,3000,0,3000
3,4000,1,0
4,6000,0,6000


* $gain = TP·TP\_value + TN·TN\_value + FP·FP\_value + FN·FN\_value$
* $gain = TP·TP\_value(0) + TN·TN\_value(0.02*amount) + FP·FP\_value(0) + FN·FN\_value(-0.8*amount)$
​
* max\_gain = représente le gain parfait du dataset

=> $score = \frac{gain}{max\_gain} ∈ [-1 ; 1]$



In [12]:
d = {'AMT_CREDIT': [1000, 2000,3000,4000,6000], 'y_actual': [0, 1, 0, 1,0]}
df = pd.DataFrame(data=d)

In [13]:
from sklearn.metrics import confusion_matrix
def performance_measure(y_actual, y_pred,df):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    list_resultat=[]
    #Etape 1 : identifier qui est qui ? 
    tn, fp, fn, tp = confusion_matrix(y_actual, y_pred).ravel()
    for i in range(len(y_pred)): 
        if y_actual[i] == y_pred[i]==1:
            TP += 1
            list_resultat.append('TP')
        if y_pred[i] == 1 and y_actual[i] == 0:
            FP += 1
            list_resultat.append('FP')
        if y_pred[i] == y_actual[i] == 0:
            TN +=1
            list_resultat.append('TN')
        if y_pred[i] == 0 and y_actual[i] == 1:
            FN +=1
            list_resultat.append('FN')
    
    #Etape 2 : transformer les types avec les coeff associés 
    df['type_matrix']=list_resultat
    df['y_actual']=y_actual
    df['max_gain']=0
    df['worst_gain']=0
    df.loc[df['y_actual']==0,'max_gain']=df.loc[df['y_actual']==0,'AMT_CREDIT']*0.02
    df.loc[df['y_actual']==0,'worst_gain']=df.loc[df['y_actual']==0,'AMT_CREDIT']*-0.8

    df.loc[df['type_matrix']=='TP','AMT_CREDIT']=df.loc[df['type_matrix']=='TP','AMT_CREDIT']*0
    df.loc[df['type_matrix']=='FP','AMT_CREDIT']=df.loc[df['type_matrix']=='FP','AMT_CREDIT']*0
    df.loc[df['type_matrix']=='TN','AMT_CREDIT']=df.loc[df['type_matrix']=='TN','AMT_CREDIT']*0.02
    df.loc[df['type_matrix']=='FN','AMT_CREDIT']=df.loc[df['type_matrix']=='FN','AMT_CREDIT']*(-0.8)
    #Etape 3 : Les formules 
    total_TP=df.loc[df['type_matrix']=='TP','AMT_CREDIT'].sum()
    total_FP=df.loc[df['type_matrix']=='FP','AMT_CREDIT'].sum()
    total_TN=df.loc[df['type_matrix']=='TN','AMT_CREDIT'].sum()
    total_FN=df.loc[df['type_matrix']=='FN','AMT_CREDIT'].sum()
    
    gain=total_TP+total_FP+total_TN+total_FN
    max_gain=df.loc[df['y_actual']==0,'max_gain'].sum()
    worst_gain=df.loc[df['y_actual']==0,'worst_gain'].sum()
    #The label is a binary variable, 0 (will repay loan on time), 1 (will have difficulty repaying loan)
    score = (gain ) / (max_gain )
    if score  <0 : 
        score=gain / (-worst_gain )
    else: 
        pass
        #score=(gain)/(worst_gain)
    print('gain',gain)
    print('max gain',max_gain)
    print('worst_gain',worst_gain)
    print('score',score)
    print('Valeurs lib:')
    print('TN:',tn)
    print('FP:',fp)
    print('FN:',fn)
    print('TP:',tp)
    print('-----------')
    print('Valeurs Man:')
    print('TN:',TN)
    print('FP:',FP)
    print('FN:',FN)
    print('TP:',TP)
    return score

In [14]:
performance_measure([0, 1, 0, 1,0], [1, 1, 1, 0,0],df)

gain -3080.0
max gain 200.0
worst_gain -8000.0
score -0.385
Valeurs lib:
TN: 1
FP: 2
FN: 1
TP: 1
-----------
Valeurs Man:
TN: 1
FP: 2
FN: 1
TP: 1


-0.385

In [14]:
y_actual=[0, 1, 0, 1]
y_pred=[1, 1, 1, 0]

In [15]:
TP = 0
FP = 0
TN = 0
FN = 0
list_resultat=[]
#Etape 1 : identifier qui est qui ? 
tn, fp, fn, tp = confusion_matrix(y_actual, y_pred).ravel()
for i in range(len(y_pred)): 
    if y_actual[i] == y_pred[i]==1:
        TP += 1
        list_resultat.append('TP')
    if y_pred[i] == 1 and y_actual[i] == 0:
        FP += 1
        list_resultat.append('FP')
    if y_pred[i] == y_actual[i] == 0:
        TN +=1
        list_resultat.append('TN')
    if y_pred[i] == 0 and y_actual[i] == 1:
        FN +=1
        list_resultat.append('FN')
    
    #Etape 2 : transformer les types avec les coeff associés 
df['type_matrix']=list_resultat    
df['max_gain']=0
df['worst_gain']=0
df.loc[df['y_actual']==0,'max_gain']=df.loc[df['y_actual']==0,'AMT_CREDIT']*0.02
df.loc[df['y_actual']==0,'worst_gain']=df.loc[df['y_actual']==0,'AMT_CREDIT']*-0.8



In [16]:
df

,AMT_CREDIT,y_actual,max_gain,type_matrix,worst_gain
0,1000,0,20.0,FP,-800.0
1,2000,1,0.0,TP,0.0
2,3000,0,60.0,FP,-2400.0
3,4000,1,0.0,FN,0.0


In [ ]:
df.loc[df['type_matrix']=='TP','AMT_CREDIT']=df.loc[df['type_matrix']=='TP','AMT_CREDIT']*0
df.loc[df['type_matrix']=='FP','AMT_CREDIT']=df.loc[df['type_matrix']=='FP','AMT_CREDIT']*0
df.loc[df['type_matrix']=='TN','AMT_CREDIT']=df.loc[df['type_matrix']=='TN','AMT_CREDIT']*0.02
df.loc[df['type_matrix']=='FN','AMT_CREDIT']=df.loc[df['type_matrix']=='FN','AMT_CREDIT']*(-0.8)
    #Etape 3 : Les formules 
total_TP=df.loc[df['type_matrix']=='TP','AMT_CREDIT'].sum()
total_FP=df.loc[df['type_matrix']=='FP','AMT_CREDIT'].sum()
total_TN=df.loc[df['type_matrix']=='TN','AMT_CREDIT'].sum()
total_FN=df.loc[df['type_matrix']=='FN','AMT_CREDIT'].sum()
    
gain=total_TP+total_FP+total_TN+total_FN
max_gain=df.loc[df['y_actual']==0,'max_gain'].sum()
worst_gain=df.loc[df['y_actual']==0,'worst_gain'].sum()
    #The label is a binary variable, 0 (will repay loan on time), 1 (will have difficulty repaying loan)
score = (gain ) / (max_gain )
    #if score => 0 : 

In [24]:
if 3 >= 1 :
    print('oui')
else : 
    print('ok' )

oui
